In [1]:
import numpy as np
import pandas as pd
import os
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')
%matplotlib inline

from PIL import Image
import tensorflow as tf
from keras.preprocessing.image import load_img
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, Input

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_df = pd.read_csv('/content/drive/MyDrive/Data/train.csv')
train_df

,nomor,jenis kelamin,usia
0,1,0,27
1,2,1,24
2,3,0,29
3,4,1,23
4,5,0,20
...,...,...,...
765,766,0,21
766,767,0,25
767,768,0,21
768,769,0,23


In [4]:
train_df1 = pd.concat([train_df]*3)
train_df1 = train_df1.sort_values(by='nomor')

In [5]:
train = train_df1.reset_index(drop=True)
train

,nomor,jenis kelamin,usia
0,1,0,27
1,1,0,27
2,1,0,27
3,2,1,24
4,2,1,24
...,...,...,...
2305,769,0,23
2306,769,0,23
2307,770,0,27
2308,770,0,27


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   nomor          2310 non-null   int64
 1   jenis kelamin  2310 non-null   int64
 2   usia           2310 non-null   int64
dtypes: int64(3)
memory usage: 54.3 KB


In [7]:
import os
path ='/content/drive/MyDrive/Data/Training'
os.listdir(path)

['766',
 '767',
 '770',
 '763',
 '761',
 '768',
 '762',
 '764',
 '769',
 '765',
 '751',
 '758',
 '757',
 '752',
 '759',
 '760',
 '756',
 '753',
 '754',
 '755',
 '743',
 '745',
 '742',
 '744',
 '747',
 '749',
 '750',
 '741',
 '746',
 '748',
 '734',
 '737',
 '733',
 '735',
 '732',
 '740',
 '738',
 '736',
 '731',
 '739',
 '727',
 '726',
 '721',
 '724',
 '725',
 '728',
 '729',
 '722',
 '730',
 '723',
 '711',
 '713',
 '716',
 '720',
 '714',
 '712',
 '719',
 '717',
 '715',
 '718',
 '707',
 '705',
 '702',
 '706',
 '703',
 '709',
 '710',
 '704',
 '708',
 '701',
 '691',
 '697',
 '698',
 '694',
 '696',
 '692',
 '700',
 '693',
 '695',
 '699',
 '688',
 '682',
 '689',
 '685',
 '684',
 '683',
 '686',
 '690',
 '687',
 '681',
 '673',
 '678',
 '676',
 '672',
 '677',
 '675',
 '674',
 '671',
 '680',
 '679',
 '661',
 '663',
 '662',
 '668',
 '665',
 '667',
 '666',
 '664',
 '669',
 '670',
 '653',
 '652',
 '658',
 '651',
 '654',
 '655',
 '660',
 '656',
 '659',
 '657',
 '642',
 '647',
 '643',
 '649',
 '641',


In [8]:
path ='/content/drive/MyDrive/Data/Training'
data_dir = os.path.join(path)

data = []
for i in range(1,771):
    for file in os.listdir(os.path.join(data_dir, str(i))):
        if file != 'desktop.ini':
          data.append(['{}/{}/{}'.format(path,str(i),file)])

data = pd.DataFrame(data, columns=['imaged_file']) 

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   imaged_file  2310 non-null   object
dtypes: object(1)
memory usage: 18.2+ KB


In [10]:
df = pd.DataFrame()
df['image_file'], df['age'] = data['imaged_file'], train['usia']
df

,image_file,age
0,/content/drive/MyDrive/Data/Training/1/1_1.jpg,27
1,/content/drive/MyDrive/Data/Training/1/1_3.jpg,27
2,/content/drive/MyDrive/Data/Training/1/1_2.jpg,27
3,/content/drive/MyDrive/Data/Training/2/2_2.jpg,24
4,/content/drive/MyDrive/Data/Training/2/2_1.jpg,24
...,...,...
2305,/content/drive/MyDrive/Data/Training/769/769_3...,23
2306,/content/drive/MyDrive/Data/Training/769/769_1...,23
2307,/content/drive/MyDrive/Data/Training/770/770_2...,27
2308,/content/drive/MyDrive/Data/Training/770/770_1...,27


In [11]:
def feature_extraction(images):
  features = []
  for image in tqdm(images):
    img = load_img(image, grayscale=True)
    img = img.resize((128,128), Image.ANTIALIAS)
    img = np.array(img)
    features.append(img)
  
  features = np.array(features)
  features = features.reshape(len(features), 128, 128, 1)
  return features  

In [12]:
X = feature_extraction(df['image_file'])

  0%|          | 0/2310 [00:00<?, ?it/s]

In [13]:
X.shape

(2310, 128, 128, 1)

In [14]:
X = X/255.0

In [15]:
y = np.array(df['age'])

In [16]:
input_shape = (128, 128, 1)

In [17]:
from keras.backend import dropout
inputs= Input(input_shape)
conv_1 = Conv2D(32, kernel_size=(3, 3), activation='relu') (inputs)
maxp_1 = MaxPooling2D(pool_size=(2, 2)) (conv_1)
conv_2 = Conv2D(64, kernel_size=(3, 3), activation='relu') (maxp_1)
maxp_2 = MaxPooling2D(pool_size=(2, 2)) (conv_2)
conv_3 = Conv2D(128, kernel_size=(3, 3), activation='relu') (maxp_2)
maxp_3 = MaxPooling2D(pool_size=(2, 2)) (conv_3)
conv_4 = Conv2D(256, kernel_size=(3, 3), activation='relu') (maxp_3)
maxp_4 = MaxPooling2D(pool_size=(2, 2)) (conv_4)

flatten = Flatten() (maxp_4)

#Fully Connected Layers
dense = Dense(256, activation='relu') (flatten)

dropout = Dropout(0.3) (dense)

output = Dense(1, activation='relu', name='age') (dropout)

model = Model(inputs=[inputs], outputs=[output])

model.compile(loss='mae', optimizer='adam', metrics='accuracy')


In [18]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856 

In [19]:
model.fit(x=X, y=y, batch_size=32, epochs=30, validation_split=0.2)

Epoch 1/30
58/58 [==============================] - 62s 1s/step - loss: 6.4649 - accuracy: 0.0000e+00 - val_loss: 6.1851 - val_accuracy: 0.0000e+00
Epoch 2/30
58/58 [==============================] - 62s 1s/step - loss: 5.0898 - accuracy: 0.0000e+00 - val_loss: 4.1882 - val_accuracy: 0.0000e+00
Epoch 3/30
58/58 [==============================] - 60s 1s/step - loss: 4.2833 - accuracy: 0.0000e+00 - val_loss: 4.8879 - val_accuracy: 0.0000e+00
Epoch 4/30
58/58 [==============================] - 62s 1s/step - loss: 4.2161 - accuracy: 0.0000e+00 - val_loss: 4.1415 - val_accuracy: 0.0000e+00
Epoch 5/30
58/58 [==============================] - 60s 1s/step - loss: 4.4008 - accuracy: 0.0000e+00 - val_loss: 3.9011 - val_accuracy: 0.0000e+00
Epoch 6/30
58/58 [==============================] - 62s 1s/step - loss: 4.1686 - accuracy: 0.0000e+00 - val_loss: 4.4736 - val_accuracy: 0.0000e+00
Epoch 7/30
58/58 [==============================] - 60s 1s/step - loss: 4.1818 - accuracy: 0.0000e+00 - val_loss

In [20]:
test_path = '/content/drive/MyDrive/Data/Testing'
list_dir = os.listdir(test_path)
print(len(list_dir))

990


In [21]:
test_path ='/content/drive/MyDrive/Data/Testing'
data_dir = os.path.join(test_path)

Testing = []
for file in os.listdir(data_dir):
  Testing.append(['{}/{}'.format(test_path,file)])

Testing = pd.DataFrame(Testing, columns=['imaged_file']) 

In [22]:
Testing

,imaged_file
0,/content/drive/MyDrive/Data/Testing/f77c0c15-7...
1,/content/drive/MyDrive/Data/Testing/ee2ff643-0...
2,/content/drive/MyDrive/Data/Testing/a978f9a2-6...
3,/content/drive/MyDrive/Data/Testing/6f1f16f5-f...
4,/content/drive/MyDrive/Data/Testing/1faa6a4b-e...
...,...
985,/content/drive/MyDrive/Data/Testing/5533a176-b...
986,/content/drive/MyDrive/Data/Testing/49a38274-5...
987,/content/drive/MyDrive/Data/Testing/4682aee3-d...
988,/content/drive/MyDrive/Data/Testing/aa14e18c-8...


In [23]:
X_test = feature_extraction(Testing['imaged_file'])

  0%|          | 0/990 [00:00<?, ?it/s]

In [24]:
X_test.shape

(990, 128, 128, 1)

In [25]:
X_test = X_test/255.0

In [26]:
y_pred = model.predict(X_test)

In [27]:
print(y_pred)

[[25.81984 ]
 [27.088894]
 [27.551779]
 [26.008106]
 [27.11836 ]
 [26.182129]
 [26.064035]
 [26.586197]
 [26.4413  ]
 [25.708702]
 [25.843204]
 [26.417889]
 [26.05216 ]
 [26.601665]
 [26.799316]
 [27.107546]
 [27.76977 ]
 [26.846994]
 [26.773394]
 [25.511759]
 [26.255943]
 [26.08116 ]
 [26.784317]
 [25.931263]
 [26.23534 ]
 [26.58247 ]
 [26.173988]
 [25.79205 ]
 [25.085773]
 [25.996376]
 [26.195675]
 [26.132053]
 [25.08307 ]
 [25.503048]
 [25.801105]
 [25.5067  ]
 [28.13941 ]
 [24.93407 ]
 [26.20146 ]
 [27.331718]
 [25.80048 ]
 [25.766571]
 [26.18763 ]
 [25.164663]
 [25.739357]
 [27.323376]
 [25.998573]
 [26.291786]
 [25.416298]
 [27.007572]
 [25.727207]
 [25.321924]
 [25.91558 ]
 [25.494452]
 [26.285414]
 [26.587658]
 [26.42673 ]
 [26.338364]
 [26.121752]
 [25.98412 ]
 [27.295128]
 [26.034874]
 [25.598629]
 [25.388014]
 [25.815105]
 [27.538486]
 [26.404331]
 [26.130793]
 [27.27075 ]
 [26.133507]
 [26.626266]
 [25.334846]
 [26.599613]
 [26.478838]
 [26.688396]
 [26.661213]
 [26.219872]

In [28]:
test_df = pd.DataFrame(y_pred, columns=['age'])

In [29]:
list_img = []
for i in list_dir:
  name = i.split('.')[0]
  list_img.append(name)
img_name = pd.DataFrame(list_img, columns=['img_file_name'])

In [30]:
Testing_df = pd.DataFrame()
Testing_df['img_file_name'], Testing_df['age'] = img_name['img_file_name'], test_df['age']

In [31]:
Testing_df

,img_file_name,age
0,f77c0c15-7dec-4cfe-bbb7-be272088d1a5,25.819839
1,ee2ff643-0bc6-438b-91e7-1f1e391e0633,27.088894
2,a978f9a2-69ec-4c89-9227-0bd0213a20db,27.551779
3,6f1f16f5-f0cf-48ef-9d95-5144c04c5399,26.008106
4,1faa6a4b-e99c-4a88-959f-ce0a780858f0,27.118361
...,...,...
985,5533a176-bea5-41cb-bcc4-938b5a104597,25.682669
986,49a38274-579b-4371-8673-cd44141ae7be,25.730846
987,4682aee3-d2f1-40c5-ab5d-44440f7630b4,26.920010
988,aa14e18c-8615-49c1-b8be-4033cd56f2a1,25.960751


In [32]:
submission = pd.read_csv('/content/drive/MyDrive/Data/submission.csv')
submission

,id,usia
0,005093b2-8c4b-4ed7-91c3-f5f4d50f8d27,NaN
1,0052554e-069e-4c43-beb0-0885e8f7684e,NaN
2,0092b954-1143-4a95-a17b-1edfa6af3b01,NaN
3,009fc28b-fe9b-441d-b8a2-ea8b7ae6ca16,NaN
4,00d0e306-06fe-45d8-ae6c-6f83ab8f7810,NaN
...,...,...
985,feb9e70f-4182-4500-866d-9b95657e727c,NaN
986,fed67ed2-620a-4ebf-b61e-16ef3d9b93f7,NaN
987,ff14c77e-c3c2-46db-9341-30de4130ef8a,NaN
988,ff662f3f-8c76-4d27-a73e-c43c4353f798,NaN


In [33]:
submission = submission.drop(columns=['usia'])

In [34]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      990 non-null    object
dtypes: object(1)
memory usage: 7.9+ KB


In [35]:
m = Testing_df[Testing_df['img_file_name']=='f77c0c15-7dec-4cfe-bbb7-be272088d1a5']['age']
print(m.iloc[0])

25.81984


In [36]:
age = []
for i in submission['id']:
  for j in Testing_df['img_file_name']:
    if i == j:
      l = Testing_df[Testing_df['img_file_name']=='{}'.format(j)]['age']
      age.append(l.iloc[0])

age = np.array(age)
age = age.reshape(len(age),1)

In [37]:
print(age)

[[25.345055]
 [26.634916]
 [28.343723]
 [25.69469 ]
 [25.570507]
 [25.635685]
 [26.820456]
 [26.065014]
 [25.687399]
 [26.129326]
 [27.683865]
 [26.280636]
 [25.824131]
 [25.96325 ]
 [25.926353]
 [26.13996 ]
 [25.435602]
 [25.931263]
 [26.466202]
 [25.319756]
 [27.16219 ]
 [26.450996]
 [27.577387]
 [26.661293]
 [26.52187 ]
 [25.517609]
 [27.0579  ]
 [27.003448]
 [26.224274]
 [25.961395]
 [28.851284]
 [27.117702]
 [25.180748]
 [24.599617]
 [25.422602]
 [26.284426]
 [26.083944]
 [26.314838]
 [26.072233]
 [25.162209]
 [26.773394]
 [27.12901 ]
 [26.137478]
 [27.421268]
 [27.365349]
 [26.152187]
 [26.147984]
 [25.549932]
 [26.268337]
 [26.587658]
 [25.949562]
 [26.285414]
 [25.287056]
 [27.008593]
 [26.419865]
 [26.898945]
 [25.01684 ]
 [26.384184]
 [27.123753]
 [25.818336]
 [27.007572]
 [25.423746]
 [26.21003 ]
 [26.564888]
 [25.183945]
 [25.632324]
 [27.18539 ]
 [26.352139]
 [25.712566]
 [27.351938]
 [25.969149]
 [27.532856]
 [25.964947]
 [25.273499]
 [25.409813]
 [26.619675]
 [26.144161]

In [38]:
m = pd.DataFrame(age, columns=['usia'])

In [39]:
submission['usia'] = m['usia']

In [40]:
submission

,id,usia
0,005093b2-8c4b-4ed7-91c3-f5f4d50f8d27,25.345055
1,0052554e-069e-4c43-beb0-0885e8f7684e,26.634916
2,0092b954-1143-4a95-a17b-1edfa6af3b01,28.343723
3,009fc28b-fe9b-441d-b8a2-ea8b7ae6ca16,25.694691
4,00d0e306-06fe-45d8-ae6c-6f83ab8f7810,25.570507
...,...,...
985,feb9e70f-4182-4500-866d-9b95657e727c,26.755287
986,fed67ed2-620a-4ebf-b61e-16ef3d9b93f7,25.895218
987,ff14c77e-c3c2-46db-9341-30de4130ef8a,25.939594
988,ff662f3f-8c76-4d27-a73e-c43c4353f798,27.009598


In [41]:
submission.to_csv('submission.csv')